**Set environment**

In [1]:
source ../config/config_duke.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /data/reddylab/Kuei
WORK DIRECTORY:     /data/reddylab/Kuei/out
CODE DIRECTORY:     /data/reddylab/Kuei/code
SING DIRECTORY:     /data/reddylab/Kuei/singularity
PATH OF SOURCE:     /data/reddylab/Kuei/source
PATH OF EXECUTABLE: /data/reddylab/Kuei/bin
PATH OF ANNOTATION: /data/reddylab/Kuei/annotation
PATH OF PROJECT:    /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS:    /data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [4]:
ASSAY=A001_K562_WSTARRseq
PREFIX=A001_K562_WSTARRseq

FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/fragment_count

IDX=0
FP_INPS=($(ls ${FD_INP}/*WGS*))
FP_INP=${FP_INPS[${IDX}]}
FN_INP=$(basename ${FP_INP})

echo ${FP_INP}
echo ${FN_INP}

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz


In [5]:
REGIONS=(GATA1    MYC       FADS)
CHROMS=( chrX     chr8      chr11)
STARTS=( 47785501 126735901 61554801)
ENDS=(   49880650 128736550 61665820)

for IDX in {0..2}; do
    echo ${IDX}
    REGION=${REGIONS[${IDX}]}
    CHROM=${CHROMS[${IDX}]}
    START=${STARTS[${IDX}]}
    END=${ENDS[${IDX}]}
    
    FN_OUT=${FN_INP/WGS/${REGION}}
    
    echo ${REGION} "|" ${CHROM} ":" ${START} "-" ${END}
    echo ${FN_OUT}
    
done

0
GATA1 | chrX : 47785501 - 49880650
A001_K562_WSTARRseq.Input.rep1.GATA1.unstranded.bed.gz
1
MYC | chr8 : 126735901 - 128736550
A001_K562_WSTARRseq.Input.rep1.MYC.unstranded.bed.gz
2
FADS | chr11 : 61554801 - 61665820
A001_K562_WSTARRseq.Input.rep1.FADS.unstranded.bed.gz


In [6]:
zcat ${FP_INP} | head -n 20

chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.
chr1	17004	17500	chr1_17004_17500	1	.
chr1	17483	17898	chr1_17483_17898	1	.
chr1	17487	17941	chr1_17487_17941	1	.
chr1	33606	33997	chr1_33606_33997	1	.
chr1	33617	33997	chr1_33617_33997	1	.
chr1	38785	39226	chr1_38785_39226	1	.
chr1	46328	46678	chr1_46328_46678	1	.
chr1	46651	47137	chr1_46651_47137	1	.
chr1	47287	47665	chr1_47287_47665	1	.
chr1	49125	49498	chr1_49125_49498	1	.
chr1	49269	49637	chr1_49269_49637	1	.
chr1	51108	51497	chr1_51108_51497	1	.
chr1	53996	54382	chr1_53996_54382	1	.
chr1	54325	54608	chr1_54325_54608	1	.
chr1	54346	54714	chr1_54346_54714	1	.
chr1	54461	55045	chr1_54461_55045	1	.
chr1	54474	54868	chr1_54474_54868	1	.


In [7]:
zcat ${FP_INP} | head -n 20 | awk '{if($3 >= 10300 && $2 <= 10500) {print}}'

chr1	10335	10549	chr1_10335_10549	1	.


In [8]:
CHROM=chr1
START=10300
END=10500
zcat ${FP_INP} |\
    head -n 20 |\
    awk -v CHROM=${CHROM} \
        -v START=${START} \
        -v STOP=${END} \
        '{if(($1== CHROM) && ($3 >= START) && ($2 <= STOP)) {print}}'

chr1	10335	10549	chr1_10335_10549	1	.


**RUN**

In [14]:
ls ${FD_RES}/results/${ASSAY}/fragment_count

A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep3.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Input.rep4.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep1.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep2.WGS.unstranded.bed.gz
A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz


In [23]:
sbatch -p ${NODE} \
    --mem 20G \
    --array 0-6 \
    -o ${FD_LOG}/extract_fragments_by_regions_WSTARR.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh
ASSAY=A001_K562_WSTARRseq
PREFIX=A001_K562_WSTARRseq

### set input and output directory
FD_INP=${FD_RES}/results/${ASSAY}/fragment_count
FD_OUT=${FD_RES}/results/${ASSAY}/fragment_count

FP_INPS=($(ls ${FD_INP}/*WGS*))
FP_INP=${FP_INPS[${SLURM_ARRAY_TASK_ID}]}
FN_INP=$(basename ${FP_INP})

REGIONS=(GATA1    MYC       FADS)
CHROMS=( chrX     chr8      chr11)
STARTS=( 47785501 126735901 61554801)
ENDS=(   49880650 128736550 61665820)

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### +++++++++++++++++++++++++++++++++++++++++++++++
### show input file
echo "Input: " ${FP_INP}
echo
echo "show first few lines of input"
zcat ${FP_INP} | head -5
echo

for IDX in {0..2}; do
    
    ### set loop variables
    REGION=${REGIONS[${IDX}]}
    CHROM=${CHROMS[${IDX}]}
    START=${STARTS[${IDX}]}
    END=${ENDS[${IDX}]}
    
    ### set input and output file
    FN_OUT=${FN_INP/WGS/${REGION}}
    FP_OUT=${FD_OUT}/${FN_OUT}
    
    ### filter out the fragments within the region specified
    zcat ${FP_INP} |\
        awk -v CHROM=${CHROM} \
            -v START=${START} \
            -v STOP=${END} \
            '{if(($1== CHROM) && ($3 >= START) && ($2 <= STOP)) {print}}' |\
        gzip -c > ${FP_OUT}
    
    ### show output file
    echo ++++++++++++++++++++++++++++++++++++++
    echo ${REGION} "|" ${CHROM} ":" ${START} "-" ${END}
    echo
    echo "Output: " ${FP_OUT}
    echo
    echo "show first few lines of output:"
    zcat ${FP_OUT} | head -n 10
    echo
    echo "show last few lines of output"
    zcat ${FP_OUT} | tail -n 10
    echo
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"
EOF

Submitted batch job 29422586


In [20]:
ls ${FD_LOG}/extract_fragments_by_regions_WSTARR.*.txt

/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/extract_fragments_by_regions_WSTARR.0.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/extract_fragments_by_regions_WSTARR.1.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/extract_fragments_by_regions_WSTARR.2.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/extract_fragments_by_regions_WSTARR.3.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/extract_fragments_by_regions_WSTARR.4.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/extract_fragments_by_regions_WSTARR.5.txt
/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log/extract_fragments_by_regions_WSTARR.6.txt


In [24]:
cat ${FD_LOG}/extract_fragments_by_regions_WSTARR.0.txt

Hostname:           x1-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         12-08-22+14:48:03

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.WGS.unstranded.bed.gz

show first few lines of input
chr1	10335	10549	chr1_10335_10549	1	.
chr1	16983	17500	chr1_16983_17500	1	.
chr1	16985	17499	chr1_16985_17499	1	.
chr1	17004	17500	chr1_17004_17500	1	.
chr1	17483	17898	chr1_17483_17898	1	.

++++++++++++++++++++++++++++++++++++++
GATA1 | chrX : 47785501 - 49880650

Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Input.rep1.GATA1.unstranded.bed.gz

show first few lines of output:
chrX	47785906	47786328	chrX_47785906_47786328	1	.
chrX	47787533	47787772	chrX_47787533_47787772	1	.
chrX	47787569	47787735	chrX_47787569_47787735	1	.
chrX	47787714	47788125	chrX_47787714_47788125	1	.
chrX	47787767	47788062	chrX_47787767_47788062	1	.
ch

In [25]:
cat ${FD_LOG}/extract_fragments_by_regions_WSTARR.6.txt

Hostname:           dl-01.genome.duke.edu
Slurm Array Index:  6
Time Stamp:         12-08-22+14:48:03

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.WGS.unstranded.bed.gz

show first few lines of input
chr1	10537	11067	chr1_10537_11067	1	.
chr1	14408	14799	chr1_14408_14799	1	.
chr1	14498	14937	chr1_14498_14937	1	.
chr1	14676	15184	chr1_14676_15184	1	.
chr1	14684	15162	chr1_14684_15162	1	.

++++++++++++++++++++++++++++++++++++++
GATA1 | chrX : 47785501 - 49880650

Output:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/A001_K562_WSTARRseq/fragment_count/A001_K562_WSTARRseq.Output.rep3.GATA1.unstranded.bed.gz

show first few lines of output:
chrX	47785062	47785649	chrX_47785062_47785649	1	.
chrX	47785094	47785628	chrX_47785094_47785628	1	.
chrX	47785156	47785640	chrX_47785156_47785640	1	.
chrX	47785290	47785672	chrX_47785290_47785672	1	.
chrX	47785381	47785723	chrX_47785381_47785723	1	.
ch